## Primera Parte - Imputación

- Se le da el dataset real correspondiente a un Banco de Japón, *japan_bank_500_conNulos.csv*. En este dataset la variable target es la y, en cuanto a las features las hay numéricas y categóricas.
- El banco no nos proveyó de una descripción de qué es cada variable (habitual en estos casos!) .  

- El dataset tiene muchos nulos.   

- Su primer tarea es imputar algún tipo de valor a los nulos de cada feature. 

Para ello:  

- Divida  hasta Validation Train y Validation Test. Cada división deberá hacerse 80/20 y con semilla 123.

- A nivel de Validation probará **distintas formas de imputar los nulos** (por ejemplo imputar media o mediana etc) y las evaluará en un **único modelo**  que será árbol con 4 niveles de profundidad en todos los casos. Tenga en cuenta que las variables categóricas deberán pasar por OneHotEncoder para ser útiles y la misma target posiblemente también. 

- La estrategia de imputación que le de mejor resultado de Accuracy con el mencionado árbol entrenando en el Validation Train y evaluando en el Validation Test será la forma de imputación elegida para la segunda parte ya que asumiremos que es la mejor para este dataset. 

En la celda siguiente escriba cuál fue la estrategia de imputación elegida y los valores de Accuracy obtenidos en cada caso. Si le resulta incómodo hacerlo aquí, puede escribir el resumen en Word. 

In [15]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [16]:
df=pd.read_csv('data/japan_bank_500_conNulos.csv')
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,y
0,b,29.83,NaN,NaN,g,NaN,v,NaN,f,f,0.0,f,NaN,216.0,0.0,-
1,NaN,36.75,125.00,y,p,NaN,v,NaN,f,f,0.0,t,g,232.0,NaN,+
2,NaN,24.92,NaN,u,g,ff,NaN,0.00,t,f,NaN,f,NaN,80.0,0.0,-
3,b,29.67,0.75,y,NaN,c,NaN,0.04,NaN,f,NaN,f,NaN,NaN,0.0,-
4,NaN,22.17,12125.00,NaN,g,NaN,v,3335.00,NaN,NaN,2.0,NaN,g,180.0,NaN,-


Separo el df en 2 partes para poder crear el train y test

In [17]:
X = df.drop(["y"], axis=1)
Y = df["y"]

In [18]:
#divido en train y test 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 123)

In [19]:
#divido en train_validation y test_validation
X_val_train, X_val_test, Y_val_train, Y_val_test = train_test_split(X_train,Y_train,test_size=0.2,random_state=123)

#### Hacemos la limpieza de datos

In [20]:
#Genero el imputador
imputador=SimpleImputer()

In [21]:
#Separo las columnas numericas y las de categoria
cols_num=X_val_train.select_dtypes(np.number).columns  
cols_cat=X_val_train.select_dtypes(include=['object']).columns  

In [22]:
t1_s=('imputador_num', SimpleImputer(strategy='median'), cols_num)  
t2_s=('imputador_cat', SimpleImputer(strategy='most_frequent'), cols_cat)  

transformador_columnas = ColumnTransformer(transformers=[t1_s,t2_s],remainder='passthrough',n_jobs=-1)  
transformador_columnas.fit(X_val_train)  
X_val_train_Transformado = transformador_columnas.transform(X_val_train) 

X_val_train_Transformado=pd.DataFrame(X_val_train_Transformado, columns=X_val_train.columns) 
X_val_train_Transformado

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,28.665,15.5,3.0,0.0,162.0,0.0,b,u,g,c,v,f,f,f,g
1,28.665,7.0,1085.0,0.0,320.0,6.0,b,u,g,c,v,f,f,f,g
2,24.58,12.5,875.0,0.0,162.0,6.0,b,u,g,c,v,t,f,f,g
3,18.83,4415.0,3.0,0.0,240.0,6.0,a,u,p,c,v,f,f,f,g
4,28.665,2.5,3.0,7.0,150.0,1270.0,b,u,g,c,v,t,t,f,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,13.75,7.0,1.75,0.0,162.0,1000.0,b,u,p,c,v,t,f,t,g
316,28.665,10.75,415.0,5.0,0.0,6.0,a,u,g,c,v,f,t,f,g
317,25.25,1.0,0.5,0.0,200.0,0.0,b,u,g,c,v,f,f,f,g
318,25.83,12835.0,0.5,0.0,0.0,6.0,b,u,g,c,v,f,f,f,g


In [23]:
X_val_test_Transformado = transformador_columnas.transform(X_val_test)
X_val_test_Transformado = pd.DataFrame(X_val_test_Transformado, columns=X_val_test.columns) 
#X_val_test_Transformado

In [24]:
# imputador kNN
t1=('imputador_num1', KNNImputer(n_neighbors=5), cols_num)  
t2=('imputador_cat1', SimpleImputer(strategy='most_frequent'), cols_cat) 

#Transformador de Columnas
transformador_columnas_knn = ColumnTransformer(transformers=[t1,t2],remainder='passthrough',n_jobs=-1)  

# fit en el Train
transformador_columnas_knn.fit(X_val_train)

# Lo aplico en el Train 
X_val_train_Transformado_knn = transformador_columnas_knn.transform(X_val_train) 
X_val_train_Transformado_knn = pd.DataFrame(X_val_train_Transformado_knn, columns=X_val_train.columns) 
X_val_train_Transformado_knn

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,26.018,15.5,44.65,0.0,374.0,0.0,b,u,g,c,v,f,f,f,g
1,27.066,75.858,1085.0,1.2,320.0,230.2,b,u,g,c,v,f,f,f,g
2,24.58,12.5,875.0,0.0,146.0,141.2,b,u,g,c,v,t,f,f,g
3,18.83,4415.0,18.65,0.0,240.0,436.8,a,u,p,c,v,f,f,f,g
4,35.268,2.5,718.9,7.0,150.0,1270.0,b,u,g,c,v,t,t,f,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,13.75,239.358,1.75,1.8,152.6,1000.0,b,u,p,c,v,t,f,t,g
316,35.118,10.75,415.0,5.0,0.0,275.4,a,u,g,c,v,f,t,f,g
317,25.25,1.0,0.5,0.0,200.0,0.0,b,u,g,c,v,f,f,f,g
318,25.83,12835.0,0.5,0.4,0.0,445.2,b,u,g,c,v,f,f,f,g


In [79]:
# Lo aplico en el Test
X_val_test_Transformado_knn = transformador_columnas_knn.transform(X_val_test) 
X_val_test_Transformado_knn = pd.DataFrame(X_val_test_Transformado_knn, columns=X_val_test.columns) 
X_val_test_Transformado_knn

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,28.7,1.5,150.4,0.0,168.0,167.8,b,u,g,i,bb,f,f,f,g
1,50.75,585.0,168.25,0.0,145.0,0.0,b,u,g,ff,v,f,f,f,g
2,51.42,5025.25,163.1,8.6,0.0,3000.0,b,u,g,c,h,f,f,f,g
3,22.33,235.216,2.0,1.4,80.0,370.8,b,u,g,c,v,f,f,f,g
4,27.034,2341.142,775.308,0.0,160.0,70.4,a,u,p,i,h,f,f,t,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,22.67,5104.258,1335.0,0.0,100.0,13.8,b,u,g,q,v,f,f,f,g
76,23.08,11.5,451.85,0.0,290.0,1.6,b,u,g,c,h,f,t,t,g
77,27.632,375.0,34.2,0.0,200.0,0.2,b,u,g,c,v,f,f,f,g
78,48.33,834.616,3.45,0.0,158.4,0.0,b,u,g,c,v,f,f,f,s


### Dummy kNN

#### Normalizador

In [64]:
X_all_knn=X_val_train_Transformado_knn.append(X_val_test_Transformado_knn,ignore_index=True)
X_all_knn

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,26.018,15.5,44.65,0.0,374.0,0.0,b,u,g,c,v,f,f,f,g
1,27.066,75.858,1085.0,1.2,320.0,230.2,b,u,g,c,v,f,f,f,g
2,24.58,12.5,875.0,0.0,146.0,141.2,b,u,g,c,v,t,f,f,g
3,18.83,4415.0,18.65,0.0,240.0,436.8,a,u,p,c,v,f,f,f,g
4,35.268,2.5,718.9,7.0,150.0,1270.0,b,u,g,c,v,t,t,f,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,22.67,5104.258,1335.0,0.0,100.0,13.8,b,u,g,q,v,f,f,f,g
396,23.08,11.5,451.85,0.0,290.0,1.6,b,u,g,c,h,f,t,t,g
397,27.632,375.0,34.2,0.0,200.0,0.2,b,u,g,c,v,f,f,f,g
398,48.33,834.616,3.45,0.0,158.4,0.0,b,u,g,c,v,f,f,f,s


In [27]:
tdummy_knn=("onehot",OneHotEncoder(sparse=False), ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15'])
t1_norm_knn=("normalizador_k",MinMaxScaler(feature_range=(0, 1)),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy_knn = ColumnTransformer(transformers=[tdummy_knn,t1_norm_knn],remainder='passthrough')

In [28]:
transformador_columnas_dummy_knn.fit(X_all_knn)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('normalizador_k', MinMaxScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [80]:
# Lo aplicamos al X_val_train
X_train_dummy_knn=transformador_columnas_dummy_knn.transform(X_val_train_Transformado_knn)
X_train_dummy_knn = X_train_dummy_knn
X_train_dummy_knn.shape

(320, 46)

In [82]:
# Lo aplicamos al X_val_test
X_test_dummy_knn=transformador_columnas_dummy_knn.transform(X_val_test_Transformado_knn)
X_test_dummy_knn = X_test_dummy_knn
X_test_dummy_knn.shape

(80, 46)

#### Estandarizador

In [31]:
tdummy=("onehot",OneHotEncoder(sparse=False), ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15'])
t1_norm_stand_knn=("estanarizador",StandardScaler(),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy_estand_knn = ColumnTransformer(transformers=[tdummy,t1_norm_stand_knn],remainder='passthrough')

In [32]:
transformador_columnas_dummy_estand_knn.fit(X_all_knn)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('estanarizador', StandardScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [83]:
# Lo aplicamos al X_val_train
X_train_dummy_stand_knn=transformador_columnas_dummy_estand_knn.transform(X_val_train_Transformado_knn)
X_train_dummy_stand_knn.shape

(320, 46)

In [84]:
# Lo aplicamos al X_val_test
X_test_dummy_stand_knn=transformador_columnas_dummy_estand_knn.transform(X_val_test_Transformado_knn)
X_test_dummy_stand_knn.shape

(80, 46)

#### Evaluamos de correccion de datos con normalizacion.


In [35]:
knn_norm=DecisionTreeClassifier(max_depth=4)
knn_norm.fit(X_train_dummy_knn,Y_val_train) 
knn_pred_norm = knn_norm.predict(X_test_dummy_knn) 

AC=accuracy_score(Y_val_test, knn_pred_norm)
F1=f1_score(Y_val_test,knn_pred_norm, average='macro')  
print("Evaluación modelo Árbol de Decisión kNN")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión kNN
Accuracy:  0.788
F1:  0.785


#### Evaluamos de correccion de datos con estandarizacion.

In [36]:
knn_stand=DecisionTreeClassifier(max_depth=4)
knn_stand.fit(X_train_dummy_stand_knn,Y_val_train) 
knn_pred_stand = knn_stand.predict(X_test_dummy_stand_knn) 

AC=accuracy_score(Y_val_test, knn_pred_stand)
F1=f1_score(Y_val_test,knn_pred_stand, average='macro')  
print("Evaluación modelo Árbol de Decisión kNN")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión kNN
Accuracy:  0.788
F1:  0.785


 ### Dummy simple

#### Normalizador

In [69]:
X_all=X_val_train_Transformado.append(X_val_test_Transformado,ignore_index=True)
X_all

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15
0,28.665,15.5,3.0,0.0,162.0,0.0,b,u,g,c,v,f,f,f,g
1,28.665,7.0,1085.0,0.0,320.0,6.0,b,u,g,c,v,f,f,f,g
2,24.58,12.5,875.0,0.0,162.0,6.0,b,u,g,c,v,t,f,f,g
3,18.83,4415.0,3.0,0.0,240.0,6.0,a,u,p,c,v,f,f,f,g
4,28.665,2.5,3.0,7.0,150.0,1270.0,b,u,g,c,v,t,t,f,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,22.67,7.0,1335.0,0.0,100.0,6.0,b,u,g,q,v,f,f,f,g
396,23.08,11.5,3.0,0.0,290.0,6.0,b,u,g,c,h,f,t,t,g
397,28.665,375.0,3.0,0.0,162.0,6.0,b,u,g,c,v,f,f,f,g
398,48.33,7.0,3.0,0.0,162.0,0.0,b,u,g,c,v,f,f,f,s


In [38]:
tdummy=("onehot",OneHotEncoder(sparse=False), ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15'])
t1_norm_simp=("normalizador",MinMaxScaler(feature_range=(0, 1)),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy = ColumnTransformer(transformers=[tdummy,t1_norm_simp],remainder='passthrough')

In [39]:
transformador_columnas_dummy.fit(X_all)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('normalizador', MinMaxScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [85]:
# Lo aplicamos al X_val_train
X_train_dummy=transformador_columnas_dummy.transform(X_val_train_Transformado)
X_train_dummy_Simple = X_train_dummy
X_train_dummy_Simple.shape

(320, 46)

In [86]:
# Lo aplicamos al X_val_test
X_test_dummy=transformador_columnas_dummy.transform(X_val_test_Transformado)
X_test_dummy_Simple = X_test_dummy
X_test_dummy_Simple.shape

(80, 46)

#### Estandarizador

In [42]:
t1_norm_stand=("estanarizador",StandardScaler(),['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])
transformador_columnas_dummy_estand = ColumnTransformer(transformers=[tdummy,t1_norm_stand],remainder='passthrough')

In [43]:
transformador_columnas_dummy_estand.fit(X_all)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot', OneHotEncoder(sparse=False),
                                 ['x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13',
                                  'x14', 'x15']),
                                ('estanarizador', StandardScaler(),
                                 ['x1', 'x2', 'x3', 'x4', 'x5', 'x6'])])

In [87]:
# Lo aplicamos al X_val_train
X_train_dummy_stand=transformador_columnas_dummy_estand.transform(X_val_train_Transformado)
X_train_dummy_Simple_stand = X_train_dummy_stand
X_train_dummy_Simple_stand.shape

(320, 46)

In [88]:
# Lo aplicamos al X_val_test
X_test_dummy_stand=transformador_columnas_dummy_estand.transform(X_val_test_Transformado)
X_test_dummy_Simple_stand = X_test_dummy_stand
X_test_dummy_Simple.shape

(80, 46)

#### Evaluamos de correccion de datos con normalizacion.

In [46]:
sim_norm=DecisionTreeClassifier(max_depth=4)
sim_norm.fit(X_train_dummy_Simple,Y_val_train) 
sim_pred_norm = sim_norm.predict(X_test_dummy_Simple) 

AC=accuracy_score(Y_val_test, sim_pred_norm)
F1=f1_score(Y_val_test,sim_pred_norm, average='macro')  
print("Evaluación modelo Árbol de Decisión dummy simple")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión dummy simple
Accuracy:  0.738
F1:  0.732


#### Evaluamos de correccion de datos con estandarizacion.

In [47]:
sim_stan=DecisionTreeClassifier(max_depth=4)
sim_stan.fit(X_train_dummy_Simple_stand,Y_val_train) 
sim_pred_stand = sim_stan.predict(X_test_dummy_Simple_stand) 

AC=accuracy_score(Y_val_test, sim_pred_stand)
F1=f1_score(Y_val_test,sim_pred_stand, average='macro')  
print("Evaluación modelo Árbol de Decisión dummy simple")
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión dummy simple
Accuracy:  0.738
F1:  0.732


## Segunda Parte:  

Una vez decidido cómo imputará deberá elegir el modelo que mejor sirve para pronosticar los valores del target.   
- Tenga en cuenta que como hará selección de modelo e hiperparámetros deberá compararlos en el Validation.     
- Una vez elegido el mejor modelo, deberá indicar cuál es la Accuracy Esperada cuando el modelo se use con nuevas observaciones.   
 En la celda siguiente indique cuál fue:  
 
 - modelo elegido:  
 - hiperparámetros:  
 - Accuracy Esperada

### Modelo arbol

In [93]:
Arb_val=DecisionTreeClassifier(max_depth=4, random_state=123)
Arb_val.fit(X_train_dummy_stand_knn,Y_val_train) 
Arv_pred = Arb_val.predict(X_test_dummy_stand_knn)
AC=accuracy_score(Y_val_test, Arv_pred)
F1=f1_score(Y_val_test,Arv_pred, average='macro') 
    

print("Evaluación modelo Árbol de Decisión")
print('profundidad del arbol: 4')
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo Árbol de Decisión
profundidad del arbol: 4
Accuracy:  0.788
F1:  0.785


### Modelo kNN

In [101]:
knn_val=neighbors.KNeighborsClassifier(n_neighbors=7,weights='uniform')
knn_val.fit(X_train_dummy_stand_knn,Y_val_train) 
knn_pred = knn_val.predict(X_test_dummy_stand_knn)
AC=accuracy_score(Y_val_test, knn_pred)
F1=f1_score(Y_val_test,knn_pred, average='macro') 
    

print("Evaluación modelo kNN")
print('Vecinos: 7')
print("Accuracy: ", round(AC,3))
print("F1: ", round(F1,3))

Evaluación modelo kNN
Vecinos: 7
Accuracy:  0.75
F1:  0.73


### Modelo regresión logística

In [50]:
reglog_val=LogisticRegression(penalty='none', random_state=123)
reglog_val.fit(X_train_dummy_stand_knn,Y_val_train)
rlv_pred=reglog_val.predict(X_test_dummy_stand_knn)
AC=accuracy_score(Y_val_test, rlv_pred)
F1=f1_score(Y_val_test,rlv_pred, average='macro')  

print("Evaluación modelo de Regresion Logistica")
print("Accuracy: ", round(AC,5))
print("F1: ", round(F1,5))

Evaluación modelo de Regresion Logistica
Accuracy:  0.725
F1:  0.72063


C:\Users\ACER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Entrenamos con todo el train. 

In [51]:
cols_n=X_train.select_dtypes(np.number).columns  
cols_c=X_train.select_dtypes(include=['object']).columns 
t1_model=('imputador_n', KNNImputer(n_neighbors= 5),cols_n)
t2_model=('imputador_c',SimpleImputer(strategy='most_frequent'),cols_c)

transformador_columnas=ColumnTransformer(transformers=[t1_model,t2_model],remainder='passthrough')
transformador_columnas.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputador_n', KNNImputer(),
                                 Index(['x2', 'x3', 'x8', 'x11', 'x14', 'x15'], dtype='object')),
                                ('imputador_c',
                                 SimpleImputer(strategy='most_frequent'),
                                 Index(['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'], dtype='object'))])

In [52]:
X_train_Trans= transformador_columnas.transform(X_train)
X_test_Trans= transformador_columnas.transform(X_test)
colum_or=cols_n.tolist() + cols_c.tolist()
X_train_Trans=pd.DataFrame(X_train_Trans, columns=colum_or)
X_test_Trans=pd.DataFrame(X_test_Trans, columns=colum_or)  

In [53]:
X_01=X_train_Trans.append(X_test_Trans,ignore_index=True)
X_01

,x2,x3,x8,x11,x14,x15,x1,x4,x5,x6,x7,x9,x10,x12,x13
0,28.0,6.5,1.46,0.2,80.0,404.2,b,u,g,c,v,t,t,f,g
1,15.83,585.0,1.5,2.0,198.4,0.0,b,u,g,c,v,t,f,f,g
2,23.284,0.0,0.0,0.0,224.0,1.0,b,y,p,c,bb,t,f,t,s
3,38.92,76.792,0.5,0.0,300.0,2.0,b,u,g,c,v,t,f,t,g
4,33.5,6358.392,7.0,0.0,280.0,1510.6,b,y,g,c,v,t,f,t,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,22.966,258.458,26.2,0.0,162.2,33.8,b,u,g,c,v,t,f,f,g
496,31.45,11.5,176.55,0.4,20.0,16.0,b,y,g,c,h,f,f,f,g
497,28.0,442.35,0.75,0.0,103.2,6397.8,b,u,g,c,v,f,f,t,g
498,34.08,6.5,125.0,0.0,126.0,0.0,b,u,g,aa,v,t,f,f,g


In [54]:
model_dummy=("onehot_m",OneHotEncoder(sparse=False), ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'])
model_estandard=("estandarizador2",StandardScaler(),['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])

trans_col1= ColumnTransformer(transformers=[model_dummy,model_estandard],remainder='passthrough')
trans_col1.fit(X_01)

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehot_m', OneHotEncoder(sparse=False),
                                 ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10',
                                  'x12', 'x13']),
                                ('estandarizador2', StandardScaler(),
                                 ['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])])

In [55]:
X_train_trans=trans_col1.transform(X_train_Trans)
X_test_trans=trans_col1.transform(X_test_Trans)

Usamos el modelo de arbol que fue el que mejor se adapto. 

In [56]:
modelo_arbol = DecisionTreeClassifier(max_depth=2, random_state=123)       
modelo_arbol.fit(X_train_trans, Y_train)              
Y_pred = modelo_arbol.predict(X_test_trans)
ACT_final=accuracy_score(Y_test, Y_pred)
F1T_final=f1_score(Y_test,Y_pred, average='macro')

print("Evaluación del modelo Seleccionado: ")
print("Accuracy: ", round(ACT_final,3))
print("F1: ", round(F1T_final,3))

Evaluación del modelo Seleccionado: 
Accuracy:  0.72
F1:  0.714


## Tercera Parte:   

Ahora deberá generar el modelo para Producción.   

- En base a lo anterior cree el modelo para producción.  

- Empléelo para pronosticar los valores de y para el dataset *japan_bank_190_X_conNulos_.csv*  que tiene 190 observaciones nuevas.    

- Debe guardar los pronósticos en un dataFrame de Pandas de una sola columna exportado a formato csv. El nombre de la columna debe ser y_pred. El nombre del Archivo debe ser SuApellido_SuNombre.csv y deberá entregarlo junto con el desarrollo completo del Parcial.

## Armamos el modelo de produccion. 

In [57]:
#Limpieza de datos
cols_nP=X.select_dtypes(np.number).columns  
cols_cP=X.select_dtypes(include=['object']).columns 
t1_modelP=('imputador_n', KNNImputer(n_neighbors= 5),cols_nP)
t2_modelP=('imputador_c',SimpleImputer(strategy='most_frequent'),cols_cP)

transformador_columnas=ColumnTransformer(transformers=[t1_modelP,t2_modelP],remainder='passthrough')
transformador_columnas.fit(X)

ColumnTransformer(remainder='passthrough',
                  transformers=[('imputador_n', KNNImputer(),
                                 Index(['x2', 'x3', 'x8', 'x11', 'x14', 'x15'], dtype='object')),
                                ('imputador_c',
                                 SimpleImputer(strategy='most_frequent'),
                                 Index(['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'], dtype='object'))])

In [58]:
#Estandarizamos y normalizamos. 
X_TransP1= transformador_columnas.transform(X)

colum_or=cols_n.tolist() + cols_c.tolist()
X_TransP1=pd.DataFrame(X_TransP1, columns=colum_or)

model_dummyP=("onehot_m",OneHotEncoder(sparse=False), ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'])
model_estandardP=("estandarizador2",StandardScaler(),['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])

trans_col1P= ColumnTransformer(transformers=[model_dummyP,model_estandardP],remainder='passthrough')
trans_col1P.fit(X_TransP1)
 
X_TransP=trans_col1P.transform(X_TransP1)
X_TransP=pd.DataFrame(X_TransP) 

In [59]:
#Creo el modelo  y lo entreno
modelo_prod = DecisionTreeClassifier(max_depth=2, random_state=123) 
modelo_prod.fit(X_TransP, Y)

DecisionTreeClassifier(max_depth=2, random_state=123)

In [60]:
#Cargo el otro df test para evaluar 
df2=pd.read_csv('data/japan_bank_190_X_conNulos_.csv')
cols_num=X.select_dtypes(np.number).columns  
cols_cat=X.select_dtypes(include=['object','bool']).columns  
t1df2=('imputador_num', KNNImputer(n_neighbors= 5),cols_num)
t2df2=('imputador_cat',SimpleImputer(strategy='most_frequent'),cols_cat)

In [61]:
#Primero lo corrigo. 
transformador_columnas=ColumnTransformer(transformers=[t1df2,t2df2],remainder='passthrough')
transformador_columnas.fit(X)
df2_Transformado= transformador_columnas.transform(df2)

cols_orden=cols_num.tolist() + cols_cat.tolist()
df2_Transformado=pd.DataFrame(df2_Transformado, columns=cols_orden)


t_dummy=("onehot2",OneHotEncoder(sparse=False), ['x1', 'x4', 'x5', 'x6', 'x7', 'x9', 'x10', 'x12', 'x13'])
t_estandard=("estandarizador2",StandardScaler(),['x2', 'x3', 'x8', 'x11', 'x14', 'x15'])
transformador_columnas_tKnnE= ColumnTransformer(transformers=[t_dummy,t_estandard],remainder='passthrough')
transformador_columnas_tKnnE.fit(X_TransP1)

df2_t=transformador_columnas_tKnnE.transform(df2_Transformado)
df2_t=pd.DataFrame(df2_t)
df2_t

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-0.116957,-0.398134,-0.325672,-0.022593,-0.216859,-0.355382
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.233789,-0.357907,-0.164035,0.038714,0.962695,-0.398283
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.986905,-0.370948,-0.157855,-0.635667,0.384954,-0.374568
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.641589,1.625555,-0.321515,-0.635667,1.618671,-0.331666
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,-0.202443,-0.396906,-0.293773,-0.635667,-1.179760,0.148866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,-0.680998,-0.270330,-0.322431,-0.635667,-0.036315,2.413232
186,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,1.280158,-0.398728,-0.082700,0.897017,0.096084,-0.091844
187,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.097807,-0.397860,4.329199,-0.635667,0.348845,-0.397839
188,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,-0.707189,-0.317292,-0.325178,3.042773,-0.457584,-0.146470


In [62]:
ypred=modelo_prod.predict(df2_t)

In [63]:
ypred= pd.DataFrame(ypred,columns=['ypred'])
ypred.to_csv('Rios_Leandro.csv',index=False)